In [15]:
from scipy.odr              import Model, Data, RealData, ODR
from scipy.stats            import linregress
from scipy.optimize         import curve_fit
from scipy.spatial.distance import squareform
from matplotlib             import pyplot as plt
from sklearn.linear_model   import HuberRegressor
from copy                   import deepcopy
from collections            import Counter
from scipy.stats            import pearsonr
import igraph  as ig
import numpy   as np
import seaborn as sns
import pandas  as pd
import random
import os
import subprocess
import re
import ete3

## Good reads:
- https://towardsdatascience.com/total-least-squares-in-comparison-with-ols-and-odr-f050ffc1a86a
- https://towardsdatascience.com/linear-regression-in-the-wild-335723a687e8
- https://en.wikipedia.org/wiki/Deming_regression
- https://en.wikipedia.org/wiki/Total_least_squares
- https://stackoverflow.com/questions/44638882/estimate-the-standard-deviation-of-fitted-parameters-in-scipy-odr
- https://www.astro.rug.nl/software/kapteyn/kmpfittutorial.html#fitting-data-when-both-variables-have-uncertainties
- https://stats.stackexchange.com/a/461968

In [3]:
class cd:
    """
    Context manager for changing the current working directory
    """
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [ ]:
def cles(lessers, greaters):
    """Common-Language Effect Size
    Probability that a random draw from `greater` is in fact greater
    than a random draw from `lesser`.
    Args:
      lesser, greater: Iterables of comparables.
      
      https://github.com/ajschumacher/cles/blob/master/cles.py
    """
    if len(lessers) == 0 and len(greaters) == 0:
        raise ValueError('At least one argument must be non-empty')
    # These values are a bit arbitrary, but make some sense.
    # (It might be appropriate to warn for these cases.)
    if len(lessers) == 0:
        return 1
    if len(greaters) == 0:
        return 0
    numerator = 0
    lessers, greaters = sorted(lessers), sorted(greaters)
    lesser_index = 0
    for greater in greaters:
        while lesser_index < len(lessers) and lessers[lesser_index] < greater:
            lesser_index += 1
        numerator += lesser_index  # the count less than the greater
    denominator = len(lessers) * len(greaters)
    return float(numerator) / denominator

In [4]:
def line(x, slope):
    """Basic linear regression 'model'"""
    return (slope * x) + 0

In [5]:
def estimate_weights(x, y, weight_estimation='gm'):
    if weight_estimation == 'gm':
        slope = np.std(y)/np.std(x)
        x_res = abs(x - line(y, 
                             slope))
        y_res = abs(y - line(x, 
                             slope))

    elif weight_estimation == 'huber':
        huber_xy  = HuberRegressor(fit_intercept=False).fit(x.reshape(-1, 1), y)
        huber_yx  = HuberRegressor(fit_intercept=False).fit(y.reshape(-1, 1), x)

        y_res     = abs(y - line(x, 
                                 huber_xy.coef_))

        x_res     = abs(x - line(y, 
                                 huber_yx.coef_))
        
    elif weight_estimation == 'ols':
        xy_params = curve_fit(line, x, y)
        y_res     = abs(y - line(x, 
                                 xy_params[0]))
        
        yx_params = curve_fit(line, y, x)
        x_res     = abs(x - line(y, 
                                 yx_params[0]))
    else:
        raise Exception('weight_estimation must be "gm", "huber", or "ols"')

    #
    # if residuals are equal do zero it drives the weight to infinity,
    #     and it is good practice not weigh things infinitely
    x_res[x_res==0] = 1e-10
    y_res[y_res==0] = 1e-10
    return(1/abs(x_res), 
           1/abs(y_res))

In [6]:
def run_odr(x, y, x_weights, y_weights):
    mod = Model(line)
    dat = Data(x, 
               y, 
               wd=x_weights, 
               we=y_weights
    )
    odr = ODR(dat, 
              mod,
              beta0=[np.std(y)/np.std(x)])
    return(odr.run())

In [7]:
def run_dist_matrix(aln_file=None, iqtree_path='iqtree', num_threads=1):
    path     = '/'.join(aln_file.split('/')[:-1])
    filename = aln_file.split('/')[-1]
    
    with cd(path):
        
        if not os.path.isfile(f'{filename}.mldist'):
            subprocess.call([iqtree_path, 
                             '-s',     filename, 
                             '-m',     'LG+G', 
#                              '-te',    'BIONJ',
                             '-nt',    'AUTO',
                             '-ntmax', str(num_threads),
                             '-keep-ident', '-safe', '-quiet'])
        
        dist_matrix = pd.read_csv(f'{filename}.mldist', 
                                  delim_whitespace = True, 
                                  skiprows         = 1, 
                                  header           = None,
                                  index_col        = 0)
        dist_matrix.columns = dist_matrix.index
    
    return(dist_matrix)

In [8]:
def balance_matrices_no_genes(matrix1, matrix2):
    
    shared_genomes = np.intersect1d(matrix1.index, 
                                    matrix2.index)
    
    matrix1 = matrix1.reindex(index  =shared_genomes, 
                              columns=shared_genomes, 
                              copy   =True)
    matrix2 = matrix2.reindex(index  =shared_genomes, 
                              columns=shared_genomes, 
                              copy   =True)
        
    return(matrix1, matrix2)

In [ ]:
def match_copies(matrix1, matrix2, taxa1, taxa2, single_copy=False):
    
    all_taxa_pairs           = pd.DataFrame()
    all_taxa_pairs['taxon1'] = [re.sub('\|\d$', '', taxon, flags=re.M)
                                for taxon in taxa1.taxon]
    all_taxa_pairs['taxon2'] = [re.sub('\|\d$', '', taxon, flags=re.M)
                                for taxon in taxa2.taxon]

    triu_indices = np.triu_indices_from(matrix1, k=1)
    condensed1   = matrix1.values[triu_indices]
    condensed2   = matrix2.values[triu_indices]

    model = Model(line)
    data  = Data(condensed1, 
                 condensed2)
    odr   = ODR(data, 
                model,
                beta0=[np.std(condensed2) /\
                       np.std(condensed1)]
               )

    regression = odr.run()

    residual_df = pd.DataFrame(columns=['x_taxon1',   'x_genome1', 
                                        'x_taxon2',   'x_genome2', 

                                        'y_taxon1',   'y_genome1', 
                                        'y_taxon2',   'y_genome2', 

                                        'x_residual', 'y_residual'],
                               data   =zip(matrix1.index[triu_indices[0]],
                                           taxa1.iloc[   triu_indices[0], 1].values,
                                           
                                           matrix1.index[triu_indices[1]],
                                           taxa1.iloc[   triu_indices[1], 1].values,

                                           matrix2.index[triu_indices[0]],
                                           taxa2.iloc[   triu_indices[0], 1].values,
                                           
                                           matrix2.index[triu_indices[1]],
                                           taxa2.iloc[   triu_indices[1], 1].values,

                                           abs(regression.delta),
                                           abs(regression.eps))
                              )
    residual_df['residual_total'] = residual_df.x_residual + residual_df.y_residual

    within_genomes = ((residual_df.x_genome1 == residual_df.x_genome2) | 
                      (residual_df.y_genome1 == residual_df.y_genome2))

    residual_df.drop(index=residual_df.index[within_genomes], inplace=True)
    
    for genome in taxa1.genome[taxa1.genome.duplicated()].unique():
    
        matrix1_homologs = taxa1.loc[taxa1.genome==genome, 
                                     'taxon'].values
        matrix2_homologs = taxa2.loc[taxa2.genome==genome, 
                                     'taxon'].values

        homolog_combinations = pd.DataFrame(columns=['homolog1', 
                                                     'homolog2', 
                                                     'residual_sum'])
        for homolog1, homolog2 in itertools.product(matrix1_homologs,
                                                    matrix2_homologs):
            tmp_df = residual_df.query('(x_taxon1 == @homolog1 | x_taxon2 == @homolog1) &'
                                       '(y_taxon1 == @homolog2 | y_taxon2 == @homolog2)')

            if not tmp_df.shape[0]:
                continue

            homolog1 = re.sub('\|\d$', 
                              '',
                              homolog1, 
                              flags=re.M)
            homolog2 = re.sub('\|\d$',
                              '', 
                              homolog2, 
                              flags=re.M)

            homolog_combinations = homolog_combinations.append(
                pd.Series(data=[homolog1, 
                                homolog2, 
                                tmp_df.residual_total.sum()],
                          index=['homolog1', 
                                 'homolog2', 
                                 'residual_sum']), 
                ignore_index=True
            )

        homolog_combinations.sort_values('residual_sum', inplace=True)
        best_pairs = set()
        
        if single_copy:
            first_row = homolog_combinations.iloc[0]
            best_pairs.add((first_row.homolog1, first_row.homolog2))
        else:
            while homolog_combinations.shape[0]:
                first_row = homolog_combinations.iloc[0]
                best_pairs.add((first_row.homolog1, first_row.homolog2))
                homolog_combinations = homolog_combinations.query(f'(homolog1 != "{first_row.homolog1}") & '
                                                                  f'(homolog2 != "{first_row.homolog2}")').copy()
            
        for homolog1, homolog2 in best_pairs:
            indices_to_drop = all_taxa_pairs.query(
                '(taxon1 == @homolog1 & taxon2 != @homolog2) |'
                '(taxon1 != @homolog1 & taxon2 == @homolog2)'
            ).index

            all_taxa_pairs.drop(index=indices_to_drop, 
                                inplace=True)

            taxa1.drop(index  =indices_to_drop, 
                       inplace=True)
            taxa2.drop(index  =indices_to_drop, 
                       inplace=True)
    
    matrix1 = matrix1.reindex(index  =taxa1.taxon, 
                              columns=taxa1.taxon, 
                              copy   =True)
    matrix2 = matrix2.reindex(index  =taxa2.taxon, 
                              columns=taxa2.taxon, 
                              copy   =True)
    
    return(matrix1, taxa1, matrix2, taxa2)

In [ ]:
def restrict_to_single_copy(matrix1, matrix2, taxa1, taxa2):
    
    all_taxa_pairs           = pd.DataFrame()
    all_taxa_pairs['taxon1'] = [re.sub('\|\d$', '', taxon, flags=re.M)
                                for taxon in taxa1.taxon]
    all_taxa_pairs['taxon2'] = [re.sub('\|\d$', '', taxon, flags=re.M)
                                for taxon in taxa2.taxon]

    triu_indices = np.triu_indices_from(matrix1, k=1)
    condensed1   = matrix1.values[triu_indices]
    condensed2   = matrix2.values[triu_indices]

    model = Model(line)
    data  = Data(condensed1, 
                 condensed2)
    odr   = ODR(data, 
                model,
                beta0=[np.std(condensed2) /\
                       np.std(condensed1)]
               )

    regression = odr.run()

    residual_df = pd.DataFrame(columns=['x_taxon1',   'x_genome1', 
                                        'x_taxon2',   'x_genome2', 

                                        'y_taxon1',   'y_genome1', 
                                        'y_taxon2',   'y_genome2', 

                                        'x_residual', 'y_residual'],
                               data   =zip(matrix1.index[triu_indices[0]],
                                           taxa1.iloc[   triu_indices[0], 1].values,
                                           
                                           matrix1.index[triu_indices[1]],
                                           taxa1.iloc[   triu_indices[1], 1].values,

                                           matrix2.index[triu_indices[0]],
                                           taxa2.iloc[   triu_indices[0], 1].values,
                                           
                                           matrix2.index[triu_indices[1]],
                                           taxa2.iloc[   triu_indices[1], 1].values,

                                           abs(regression.delta),
                                           abs(regression.eps))
                              )
    residual_df['residual_total'] = residual_df.x_residual + residual_df.y_residual

    within_genomes = ((residual_df.x_genome1 == residual_df.x_genome2) | 
                      (residual_df.y_genome1 == residual_df.y_genome2))

    residual_df.drop(index=residual_df.index[within_genomes], inplace=True)
    
    for genome in taxa1.genome[taxa1.genome.duplicated()].unique():
    
        matrix1_homologs = taxa1.loc[taxa1.genome==genome, 
                                     'taxon'].values
        matrix2_homologs = taxa2.loc[taxa2.genome==genome, 
                                     'taxon'].values

        homolog_combinations = pd.DataFrame(columns=['homolog1', 
                                                     'homolog2', 
                                                     'residual_sum'])
        for homolog1, homolog2 in itertools.product(matrix1_homologs,
                                                    matrix2_homologs):
            tmp_df = residual_df.query('(x_taxon1 == @homolog1 | x_taxon2 == @homolog1) &'
                                       '(y_taxon1 == @homolog2 | y_taxon2 == @homolog2)')

            if not tmp_df.shape[0]:
                continue

            homolog1 = re.sub('\|\d$', 
                              '',
                              homolog1, 
                              flags=re.M)
            homolog2 = re.sub('\|\d$',
                              '', 
                              homolog2, 
                              flags=re.M)

            homolog_combinations = homolog_combinations.append(
                pd.Series(data=[homolog1, 
                                homolog2, 
                                tmp_df.residual_total.sum()],
                          index=['homolog1', 
                                 'homolog2', 
                                 'residual_sum']), 
                ignore_index=True
            )

        homolog_combinations.sort_values('residual_sum', inplace=True)
        best_pairs = set()
        first_row  = homolog_combinations.iloc[0]
        best_pairs.add((first_row.homolog1, first_row.homolog2))
            
        for homolog1, homolog2 in best_pairs:
            indices_to_drop = all_taxa_pairs.query(
                '(taxon1 == @homolog1 & taxon2 != @homolog2) |'
                '(taxon1 != @homolog1 & taxon2 == @homolog2)'
            ).index

            all_taxa_pairs.drop(index=indices_to_drop, 
                                inplace=True)

            taxa1.drop(index  =indices_to_drop, 
                       inplace=True)
            taxa2.drop(index  =indices_to_drop, 
                       inplace=True)
    
    matrix1 = matrix1.reindex(index  =taxa1.taxon, 
                              columns=taxa1.taxon, 
                              copy   =True)
    matrix2 = matrix2.reindex(index  =taxa2.taxon, 
                              columns=taxa2.taxon, 
                              copy   =True)
    
    return(matrix1, taxa1, matrix2, taxa2)

In [9]:
def balance_matrices(matrix1, matrix2,  gene_sep='_', single_copy=False):
    
    if gene_sep == '_':
        regex = re.compile('^(GC[AF]_\d+(?:\.\d)?)[_|](.*)$')
    elif gene_sep == '.':
        regex = re.compile('^(\d+?)\.(.*)$')
    
    tmp_taxa = []
    for index in matrix1.index:
        genome, gene = re.search(regex, index).groups()
        tmp_taxa.append([index, genome, gene])

    taxa1 = pd.DataFrame(columns=['taxon', 'genome', 'gene'],
                         data   =tmp_taxa)

    tmp_taxa = []
    for index in matrix2.index:
        genome, gene = re.search(regex, index).groups()
        tmp_taxa.append([index, genome, gene])

    taxa2 = pd.DataFrame(columns=['taxon', 'genome', 'gene'],
                         data=tmp_taxa)

    shared_genomes = np.intersect1d(taxa1.genome.unique(), 
                                    taxa2.genome.unique())

    taxa1 = taxa1[taxa1.genome.isin(shared_genomes)]
    taxa2 = taxa2[taxa2.genome.isin(shared_genomes)]

    if not taxa1.genome.is_unique or not taxa2.genome.is_unique:
    
        taxa1_frequency = taxa1.genome.value_counts() 
        taxa2_frequency = taxa2.genome.value_counts() 

        for genome in shared_genomes:
            genome1_count = taxa1_frequency[genome]
            genome2_count = taxa2_frequency[genome]

            if genome1_count > 1:
                #
                # one of the matrices must be traversed in the inversed order to make sure an 
                #     all VS all combination is obtained. That is the reason of the "iloc[::-1]"
                #     during the querying
                tmp_df = taxa2.iloc[::-1].query('genome == @genome').copy()
                for _ in range(genome1_count - 1):
                    for index, row in tmp_df.iterrows():
                        tmp_row = row.copy()
                        tmp_row.taxon += f'|{_}'
                        taxa2      = taxa2.append(tmp_row, ignore_index=True)

                        reference_name = re.sub('\|\d+$', '', tmp_row.taxon, flags=re.M)
                        matrix2[    tmp_row.taxon] = matrix2[    reference_name]
                        matrix2.loc[tmp_row.taxon] = matrix2.loc[reference_name]


            if genome2_count > 1:
                #
                # as we queried the other matrix in the reverse order, we traverse this one regularly
                tmp_df = taxa1.query('genome == @genome').copy()
                for _ in range(genome2_count - 1):
                    for index, row in tmp_df.iterrows():
                        tmp_row = row.copy()
                        tmp_row.taxon += f'|{_}'
                        taxa1 = taxa1.append(tmp_row, ignore_index=True)

                        reference_name = re.sub('\|\d+$', '', tmp_row.taxon, flags=re.M)
                        matrix1[    tmp_row.taxon] = matrix1[    reference_name]
                        matrix1.loc[tmp_row.taxon] = matrix1.loc[reference_name]

    taxa1.sort_values('genome', inplace=True)
    taxa2.sort_values('genome', inplace=True)

    taxa1.reset_index(drop=True, inplace=True)
    taxa2.reset_index(drop=True, inplace=True)
        
    all_taxa_pairs           = pd.DataFrame()
    all_taxa_pairs['taxon1'] = [re.sub('\|\d$', '', taxon, flags=re.M)
                                for taxon in taxa1.taxon]
    all_taxa_pairs['taxon2'] = [re.sub('\|\d$', '', taxon, flags=re.M)
                                for taxon in taxa2.taxon]
    
    matrix1 = matrix1.reindex(index  =taxa1.taxon, 
                              columns=taxa1.taxon, 
                              copy   =True)
    matrix2 = matrix2.reindex(index  =taxa2.taxon, 
                              columns=taxa2.taxon, 
                              copy   =True)
    
    if not taxa1.genome.is_unique or not taxa2.genome.is_unique:
        matrix1, taxa1, matrix2, taxa2 = match_copies(matrix1, matrix2, taxa1, taxa2, single_copy=single_copy)
    
    return(matrix1, taxa1, matrix2, taxa2)

In [ ]:
def get_matrix_from_tree(tree):
    leaf_names = []
    for count, node in enumerate(tree.traverse()):
        if not node.is_leaf():
            node.name = 'node_%i' % count
        else:
            leaf_names.append(node.name)

    edges = []
    for node in tree.traverse():
        if not node.is_leaf():
            for child in node.get_children():
                edges.append((node.name,
                              child.name,
                              child.dist))

    dag  = ig.Graph.TupleList(edges     =tuple(edges), 
                              directed  =False,
                              edge_attrs=['weight']
                             )
    
    patristic_distances     = np.array(dag.shortest_paths(source=leaf_names, 
                                                          target=leaf_names, 
                                                          weights='weight'))
                                       
    np.fill_diagonal(patristic_distances, 0.0)
    
    dist_matrix = pd.DataFrame(index  =leaf_names, 
                               columns=leaf_names, 
                               data   =patristic_distances
                              )
    return(dist_matrix)

In [10]:
def assess_coevolution(matrix1, matrix2, 
                       pearson=False, geneIDs=True,
                       gene_sep='_', single_copy=False):
    if geneIDs:
        matrix1, taxa1, matrix2, taxa2 = balance_matrices(matrix1.copy(), matrix2.copy(), gene_sep, single_copy=single_copy)
    else:
        matrix1, matrix2 = balance_matrices_no_genes(matrix1.copy(), matrix2.copy())

    if matrix1.shape[0] < 10:
        return([None, None])

    condensed1 = squareform(matrix1.values, checks=False)
    condensed2 = squareform(matrix2.values, checks=False)
    
    if pearson:
        return(pearsonr(condensed1,
                        condensed2))
    
    odr_weights = estimate_weights(condensed1, condensed2)
    
    regression = run_odr(condensed1, 
                         condensed2, 
                         *odr_weights)
    
    mean_x = np.mean(condensed1)
    mean_y = np.mean(condensed2)

    mean_pred_x = regression.xplus.mean()
    mean_pred_y = regression.y.mean()

    x_SSres = sum(regression.delta**2)
    y_SSres = sum(regression.eps  **2)
    SSres   = x_SSres + y_SSres

    x_SSreg = sum(
        (regression.xplus - mean_pred_x)**2
    )
    y_SSreg = sum(
        (regression.y     - mean_pred_y)**2
    )
    SSreg   = x_SSreg + y_SSreg

    x_SStot = sum(
        (condensed1 - mean_x)**2
    )
    y_SStot = sum(
        (condensed2 - mean_y)**2
    )
    SStot   = x_SStot + y_SStot

    r2 = 1 - SSres/SStot
#     r2 = SSreg/SStot
    
    return(regression, r2)

In [9]:
# dist1 = run_dist_matrix('/work/clusterEvo/distance_matrices/000284/000284')
# dist2 = run_dist_matrix('/work/clusterEvo/distance_matrices/000302/000302')

# regression, r2 = assess_coevolution(dist1, dist2)

# regression.pprint()
# print(f'\nR**2 = {r2}')

## Beta: [0.49615544]
## Beta Std Error: [0.00033644]
## Beta Covariance: [[1.99398167e-06]]
## Residual Variance: 0.056766828371899364
## Inverse Condition #: 1.0
## Reason(s) for Halting:
##   Sum of squares convergence
##
## R**2 = 0.8947962483462916

Beta: [0.49615544]
Beta Std Error: [0.00033644]
Beta Covariance: [[1.99398167e-06]]
Residual Variance: 0.056766828371899364
Inverse Condition #: 1.0
Reason(s) for Halting:
  Sum of squares convergence

R**2 = 0.8947962483462916
